In [1]:
import av
import pandas as pd
from av.video.stream import VideoStream
import numpy as np
import cv2
import os
from tqdm import tqdm_notebook

from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier

In [3]:
res = pd.read_csv('trainset/train.txt' , sep = ' ' , header=None, dtype=str)
x = []
for a in res[1]:
    x.append(list(a))
res.drop([1] , axis = 1 , inplace=True)
res = pd.concat([res , pd.DataFrame(x , dtype=int)] , axis=1)
res.columns = ['file_name' , 'bridge' , 'city_entry' , 'city_exit' , 'road_bump' , 'screen_wipers' , 'zebra']
res.head()

,file_name,bridge,city_entry,city_exit,road_bump,screen_wipers,zebra
0,akn.022.008.left.avi,0,0,0,1,0,0
1,akn.031.029.left.avi,0,0,0,1,0,1
2,akn.031.037.left.avi,0,0,0,1,1,1
3,akn.078.031.left.avi,0,0,1,0,0,0
4,akn.082.013.left.avi,0,0,1,0,0,0


In [4]:
def get_stats_for_file(array_file):
    array_file = np.array(array_file)
    
    minim_mean = array_file.mean() - array_file.min()
    maxim_mean = array_file.max() - array_file.mean()
    std = array_file.std()
    
    diff = np.diff(array_file , n = 1)
    min_max_1 = diff.max() - diff.min()
    std_diff_1 = diff.std()
    
    diff = np.diff(array_file , n = 2)
    min_max_2 = diff.max() - diff.min()
    std_diff_2 = diff.std()
    
    diff = np.diff(array_file , n = 3)
    min_max_3 = diff.max() - diff.min()
    std_diff_3 = diff.std()
    
    return [minim_mean , maxim_mean  ,  std,  min_max_1 ,  std_diff_1 , min_max_2 ,  std_diff_2 ,min_max_3 ,  std_diff_3]

def get_pixel_point(image):
    image = image[0:100,:]
    return image[:,:,0].mean()*0.299 + image[:,:,1].mean()*0.587 + image[:,:,2].mean()*0.114

def get_array(path):
    files = [video for video in os.listdir(path) if ".avi" in video and "._" not in video]
    data = []
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([15, 15, 30])

    kernel = np.ones((5,5),'int')
    for file in tqdm_notebook(files):
        array_1 = []
        array_2 = []
        array_3 = []
        input_container = av.open(path + file)
        input_packets = input_container.demux()
        for packet in input_packets:
            frames = packet.decode()
            for raw_frame in frames:
                x = raw_frame.to_rgb().to_nd_array()
                
                array_1.append(x[:,:,0].mean()*0.299  + x[:,:,1].mean()*0.587 + x[:,:,2].mean()*0.114)
                array_2.append(get_pixel_point(x))
                
                frame_hsv = cv2.cvtColor(x, cv2.COLOR_RGB2HSV)
                black_mask = cv2.inRange(frame_hsv, lower_black, upper_black)
                black_mask = cv2.dilate(black_mask,kernel)
                array_3.append((black_mask==255).sum()/black_mask.size)
        data.append([file , array_1 , array_2 , array_3])
    return data

In [7]:
def generate_fich(path):
    fich = []
    data =  get_array(path)
    file = data[0]
    for x in data[1:]:
        fich.append(get_stats_for_file(x))
    fich = np.hstack(fich)
    return pd.DataFrame([data , fich] , dtype=float)

In [8]:
def get_fich(data):
    fich = []
    for x in data:
        fich.append(np.hstack([x[0] , get_stats_for_file(x[1]) , get_stats_for_file(x[2]) ,  get_stats_for_file(x[3])]))
    return pd.DataFrame(fich)

In [9]:
train = get_array('trainset/')
valid = get_array('validationset/')

Stream #0: not enough frames to estimate rate; consider increasing probesize


In [10]:
train_fich = get_fich(train)
valid_fich = get_fich(valid)

In [11]:
def get_right_answer(data , res , col):
    r = []
    for i in range(data.shape[0]):
        r.append(int(res[res.file_name == data.loc[i][0]][col]))
    return np.array(r)

In [18]:
path_valid = 'validationset/'
files_valid = [video for video in os.listdir(path_valid) if ".avi" in video and "._" not in video]
sub = pd.DataFrame(files_valid)

dict_answ = {
    0: '000000',
    1: '100000',
    2: '010000',
    3: '001000',
    4: '000100',
    5: '000010',
    6: '000001'
}

for i , porog  in zip([1 , 5] , [0.5,0.5]):
    right_answ = get_right_answer(train_fich , res , res.columns[i])
    model = CatBoostClassifier(random_seed=0)
    a = cross_val_score(model , train_fich.drop([0] , axis=1).values, right_answ , cv = 5 , scoring='roc_auc')
    print (a.mean() , a.std())
    model.fit(train_fich.drop([0] , axis=1).values, right_answ)
    sub['res'] = model.predict_proba(valid_fich.drop([0] , axis=1).values)[:,1]
    sub['res'] = sub['res'].apply(lambda x: dict_answ[i] if x >= porog  else dict_answ[0])
    print (sub[sub.res == dict_answ[i]].shape)
    sub.to_csv(str(i) + '.txt' , sep = ' ' , header=None , index=None)

0.990938563352 0.0104668134402
(53, 2)
0.965714285714 0.0685714285714
(31, 2)


In [19]:
train_fich.to_csv('train.csv')
valid_fich.to_csv('valid.csv')